# [Lab 5] Neural end-to-end ASR <span style="color:red">implementation</span> 

Adapted from colab presented at [Interspeech 2019 tutorial](https://colab.research.google.com/github/espnet/interspeech2019-tutorial/blob/master/notebooks/interspeech2019_asr/interspeech2019_asr.ipynb), by Shigeki Karita.


Speech Processing Course, May, 2020


Instituto Superior Técnico


# A. Overview

Automatic speech recognition (ASR) as a machine learning problem is a sequence-to-sequence problem. We want to find the most probable sequence of words given input features.

ESPnet is a speech processing toolkit that provides bash recipes and python libraries for different speech processing tasks, including text-to-speech and ASR.
You can find more details on ESPnet recipes [here](https://github.com/espnet/espnet/tree/master/egs) and on pretrained models [here](https://github.com/espnet/espnet#asr-results).

**Open this colab in playground mode or save a copy to your own Google drive.**


## B. Installation

In Google colab, we can use pre-compiled binaries for fast startup (3 min):

In [1]:
# WARNING: This code takes a few minutes!
# OS setup
!cat /etc/os-release
!apt-get install -qq bc tree sox

# espnet setup
!git clone --depth 5 https://github.com/espnet/espnet
!pip install -q torch==1.5
!cd espnet; pip install -q -e .

# download pre-compiled warp-ctc and kaldi tools
!espnet/utils/download_from_google_drive.sh \
    "https://drive.google.com/open?id=13Y4tSygc8WtqzvAVGK_vRV9GlV7TRC0w" espnet/tools tar.gz > /dev/null
!cd espnet/tools/warp-ctc/pytorch_binding && \
    pip install -U dist/warpctc_pytorch-0.1.1-cp36-cp36m-linux_x86_64.whl

# make dummy activate
!mkdir -p espnet/tools/venv/bin && touch espnet/tools/venv/bin/activate
!echo "setup done."

NAME="Ubuntu"
VERSION="18.04.3 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.3 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic
Selecting previously unselected package libopencore-amrnb0:amd64.
(Reading database ... 144467 files and directories currently installed.)
Preparing to unpack .../0-libopencore-amrnb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrnb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected package libopencore-amrwb0:amd64.
Preparing to unpack .../1-libopencore-amrwb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrwb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected package libmagic-mgc.
Preparing to unpack .../2-libmagic-mgc_1%3a5.32-2ubuntu0.4_amd64.deb ...
Unpacking libmagic-mgc (1:5.32-2ubuntu0.4)

# C. ESPnet pipeline

The image shows all stages in the ASR pipeline, which are common in all recipes of ESPnet. The recipes can be found in `espnet/egs/xx/asr1/run.sh`.

![image.png](https://github.com/espnet/interspeech2019-tutorial/blob/master/notebooks/interspeech2019_asr/figs/stages.png?raw=1)

## C.1 Understand directory structure

Each recipe directory contains the file `run.sh` and the following directories:

- `conf/`: configurations for stages and machines
- `data/`: folows a style similar to [Kaldi](https://kaldi-asr.org/doc/data_prep.html) in Stage 0 - 1
- `dump/`: python-friendly dataset (e.g., json, hdf5) in Stage 2
- `exp/`: log files, saved model parameters and results in Stage 3 - 5

In this lab you will follow the an4 recipe, which uses the [Census Database](http://www.speech.cs.cmu.edu/databases/an4/). Run the following cell to inspect the structure of the an4 directory. 


In [0]:
!tree -L 1 espnet/egs/an4/asr1

espnet/egs/an4/asr1
├── cmd.sh
├── conf
├── local
├── path.sh
├── RESULTS
├── run.sh
├── steps -> ../../../tools/kaldi/egs/wsj/s5/steps
└── utils -> ../../../tools/kaldi/egs/wsj/s5/utils

4 directories, 4 files


## C.2 Data preparation (Stage 0 - 2)

Stages 0 to 2 will take care of data preparation. To run only some stages of the recipe we can use the flag `--stop-stage`.
For example, if you add `--stop-stage 2`, you can stop the script before neural network training.

These stages perform filter-bank (fbank) feature extraction from speech, normalization, and text formatting.

![image.png](https://github.com/espnet/interspeech2019-tutorial/blob/master/notebooks/interspeech2019_asr/figs/stages_prep.png?raw=1)

In [2]:
!cd espnet/egs/an4/asr1; ./run.sh  --ngpu 1 --stop-stage 2

stage -1: Data Download
local/download_and_untar.sh: downloading data (64 MB) from http://www.speech.cs.cmu.edu/databases/an4//an4_sphere.tar.gz.
--2020-06-03 16:24:47--  http://www.speech.cs.cmu.edu/databases/an4//an4_sphere.tar.gz
Resolving www.speech.cs.cmu.edu (www.speech.cs.cmu.edu)... 128.2.204.214
Connecting to www.speech.cs.cmu.edu (www.speech.cs.cmu.edu)|128.2.204.214|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64327561 (61M) [application/x-gzip]
Saving to: 'an4_sphere.tar.gz'

an4_sphere.tar.gz   100%[===================>]  61.35M  15.1MB/s    in 5.1s    

2020-06-03 16:24:53 (12.0 MB/s) - 'an4_sphere.tar.gz' saved [64327561/64327561]

local/download_and_untar.sh: line 57: cd: ./downloads: No such file or directory
an4/
an4/README
an4/etc/
an4/etc/an4_test.fileids
an4/etc/an4.ug.lm
an4/etc/an4.ug.lm.DMP
an4/etc/an4_train.fileids
an4/etc/an4_train.transcription
an4/etc/an4_test.transcription
an4/etc/an4.dic
an4/etc/an4.phone
an4/etc/an4.filler
an4/

#### JSON dataset

Stage 2 dumps the dataset of speech feature and trascription pairs into JSON format in `dump`directory. Inspect the content of data.json file with the following command.

In [0]:
!cat espnet/egs/an4/asr1/dump/train_dev/deltafalse/data.json

## C.3 Neural Network Training (Stage 3 - 4)
Now you will execute stages 3 and 4 of the script `run.sh` to train the neural network (NN).

You can configure NN training using the files `conf/train_xxx.yaml`. Tun the following cell to see the configuration files available in this recipe.

![image.png](https://github.com/espnet/interspeech2019-tutorial/blob/master/notebooks/interspeech2019_asr/figs/stages_nn.png?raw=1)

In [0]:
!tree espnet/egs/an4/asr1/conf

espnet/egs/an4/asr1/conf
├── decode_ctcweight0.5.yaml
├── decode_ctcweight1.0.yaml
├── decode_v2_ctcweight0.5.yaml
├── decode_v2_ctcweight1.0.yaml
├── fbank.conf
├── gpu.conf
├── lm_seq_rnn.yaml
├── lm_transformer.yaml
├── lm.yaml
├── pitch.conf
├── queue.conf
├── slurm.conf
├── train_mtlalpha0.5.yaml
└── train_mtlalpha1.0.yaml

0 directories, 14 files


#### Training config: RNN with Attention + CTC

We will use the mtlalpha configuration, which corresponds to an recurrent neural network (RNN) model with Connectionist Temporal Classification (CTC) (version 1.0 - train_mtlalpha1.0.yaml) or CTC and attention (version 0.5 - train_mtlalpha0.5.yaml) 

You can find a complete list of other options [here](https://espnet.github.io/espnet/apis/espnet_bin.html#Named%20Arguments) and [here](https://espnet.github.io/espnet/_modules/espnet/nets/pytorch_backend/e2e_asr.html#E2E.add_arguments).

Inspect the configuration file with the following code cell.

Note that you should use RNN+CTC/attention for question 6 of the lab, and RNN+CTC for question 7 of the lab.

In [0]:
!cat espnet/egs/an4/asr1/conf/train_mtlalpha0.5.yaml

#### Run Language Modelling (LM) and ASR NN training

Run the following cell to perform stages 3 and 4.

In [17]:
# WARNING: This code takes several minutes!
!cd espnet/egs/an4/asr1; ./run.sh  --ngpu 1 --stage 3 --stop-stage 4 --train-config ./conf/train_mtlalpha0.5.yaml

dictionary: data/lang_1char/train_nodev_units.txt
stage 3: LM Preparation
stage 4: Network Training


## C.4 Decoding and evaluation (Stage 5)

Now you will perform the last stage of the pipeline: deconding and scoring.

![image.png](https://github.com/espnet/interspeech2019-tutorial/blob/master/notebooks/interspeech2019_asr/figs/stages_eval.png?raw=1)

#### decoding YAML config

Inspect the configuration file for decoding, running the next cell.

The decoding score is computed as follows:

$\mathrm{argmax}_y (1 - \lambda) \log P_{dec}(y|x) + \lambda \log P_{ctc}(y|x) + \gamma \log P_{lm}(y) + b |y|$

In [0]:
!cat espnet/egs/an4/asr1/conf/decode_ctcweight0.5.yaml

#### Run ASR decoding

In [0]:
# WARNING: This code takes several minutes!
!cd espnet/egs/an4/asr1; source path.sh; \
  ./run.sh --stage 5 --decode-config $(change_yaml.py conf/decode_ctcweight0.5.yaml -a batchsize=0) --train-config conf/train_mtlalpha1.0.yaml

!cd espnet/egs/an4/asr1; source path.sh; \
  ./run.sh --stage 5 --decode-config $(change_yaml.py conf/decode_ctcweight1.0.yaml -a batchsize=0) --train-config conf/train_mtlalpha1.0.yaml

## C.5 Check evaluation results

The results obtained at the character level are saved in the directory `exp/(train dir)/(decode dir)/result.txt`. Run the following cells to inspect the results.

In [0]:
!ls espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/*/result.txt | xargs -n1 grep -e Avg -e SPK -m 2

In [0]:
!head -n 36 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/result.txt




                     SYSTEM SUMMARY PERCENTAGES by SPEAKER                      

,-------------------------------------------------------------------------------------------------------.
|exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/hyp.trn|
|-------------------------------------------------------------------------------------------------------|
|   SPKR     |   # Snt      # Wrd   |   Corr        Sub         Del        Ins        Err       S.Err   |
|------------+----------------------+-------------------------------------------------------------------|
|   fcaw     |     13         237   |   92.0        5.9         2.1        0.0        8.0        46.2   |
|------------+----------------------+-------------------------------------------------------------------|
|   fjlp     |     13         242   |   93.4        4.5         2.1        0.0        6.6        53.8   |
|------------+----------------------+------------------------------

In [0]:
!tail -n 7 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/result.txt

id: (mmxg-mmxg-cen8-b)
Scores: (#C #S #D #I) 26 2 8 0
REF:  O C t O B e r <space> t w e n t y <space> f o u r <space> n i n E T E e N <space> s e v e n T Y 
HYP:  E N t * * e r <space> t w e n t y <space> f o u r <space> n i n * * * e * <space> s e v e n * * 
Eval: S S   D D                                                       D D D   D                   D D 




# D. Include your own recordings in the test set

Now you will record some utterances and include them in the test set, to evaluate how the model performs. 
Follow the steps in question 9 in the lab guide, and write the necessary code below this section.


In [0]:
# 9.2 Insert your code below these lines, before running this cell,
# if you choose to perform this task with a command, or else you can do it 
# directly on the tree structure on the left.
!mkdir -p espnet/egs/an4/asr1/downloads/my_test/ricardo
!mkdir -p espnet/egs/an4/asr1/downloads/my_test/goncalo


In [5]:
# 9.3 Insert your code below this line,  before running this cell,
# if you choose to perform this task with a command, or else you can do it 
# directly on the tree structure on the left.
!mkdir -p espnet/egs/an4/asr1/data/test_backup
!cp -r espnet/egs/an4/asr1/data/test/* espnet/egs/an4/asr1/data/test_backup
!rm espnet/egs/an4/asr1/data/test/* 

rm: cannot remove 'espnet/egs/an4/asr1/data/test/conf': Is a directory


In [7]:
# 9.4 After uploading wav.scp, utt2spk and text, inspect the content if the directpry test to make sure you find the files:
# test/wav.scp test/utt2spk test/text test/conf/fbank.conf test/conf/pitch.conf
!tree espnet/egs/an4/asr1/data/test/

espnet/egs/an4/asr1/data/test/
├── conf
│   ├── fbank.conf
│   └── pitch.conf
├── text
├── utt2spk
└── wav.scp

1 directory, 5 files


In [18]:
# 9.5 Run this cell to sort the kaldi files generated, and to generate the rest of the necessary files:
!cd espnet/egs/an4/asr1; utils/fix_data_dir.sh data/test

utils/fix_data_dir.sh: file data/test/utt2spk is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file data/test/text is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file data/test/wav.scp is not in sorted order or not unique, sorting it
fix_data_dir.sh: kept all 134 utterances.
fix_data_dir.sh: old files are kept in data/test/.backup


In [15]:
# Listen to the recordings you generated. Insert the correct file_id in key=.... and then run this cell.
!pip install kaldiio

import os
import kaldiio
from IPython.display import Audio

key="ricardo_rec1"
root = "espnet/egs/an4/asr1"

try:
  d = os.getcwd()
  os.chdir(root)
  sr, wav = kaldiio.load_scp("data/test/wav.scp")[key]
finally:
  os.chdir(d)
Audio(wav, rate=sr)

In [19]:
# 9.6 Insert your code below this line, before running this cell:
!cd espnet/egs/an4/asr1; ./run.sh  --ngpu 1 --stage 1 --stop-stage 2

stage 1: Feature Generation
steps/make_fbank_pitch.sh --cmd run.pl --nj 8 --write_utt2num_frames true data/test exp/make_fbank/test fbank
utils/validate_data_dir.sh: Successfully validated data-directory data/test
steps/make_fbank_pitch.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
steps/make_fbank_pitch.sh: Succeeded creating filterbank and pitch features for test
fix_data_dir.sh: kept all 134 utterances.
fix_data_dir.sh: old files are kept in data/test/.backup
steps/make_fbank_pitch.sh --cmd run.pl --nj 8 --write_utt2num_frames true data/train exp/make_fbank/train fbank
steps/make_fbank_pitch.sh: moving data/train/feats.scp to data/train/.backup
utils/validate_data_dir.sh: Successfully validated data-directory data/train
steps/make_fbank_pitch.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
steps/make_fbank_pitch.sh: Succeeded creating filterbank and pitch features for train
fix_data_dir.sh: kept all 948 utterances.
fix_data_d

In [0]:
# 9.7 Make sure that you were able to extract features for your recordings. Replace the string "studentname" with your name.
!cat espnet/egs/an4/asr1/dump/test/deltafalse/data.json | grep "ricardo"

        "ricardo_rec1": {
            "utt2spk": "ricardo"
        "ricardo_rec2": {
            "utt2spk": "ricardo"


In [20]:
# 9.8 Insert your code below this line, before running this cell:
!cd espnet/egs/an4/asr1; source path.sh; \
  ./run.sh --stage 5 --decode-config $(change_yaml.py conf/decode_ctcweight0.5.yaml -a batchsize=0) --train-config conf/train_mtlalpha0.5.yaml

!cd espnet/egs/an4/asr1; source path.sh; \
  ./run.sh --stage 5 --decode-config $(change_yaml.py conf/decode_ctcweight1.0.yaml -a batchsize=0) --train-config conf/train_mtlalpha0.5.yaml

dictionary: data/lang_1char/train_nodev_units.txt
stage 5: Decoding
2020-06-03 17:17:12,043 (splitjson:43) INFO: /usr/bin/python3 /content/espnet/egs/an4/asr1/../../../utils/splitjson.py --parts 8 dump/train_dev/deltafalse/data.json
2020-06-03 17:17:12,045 (splitjson:55) INFO: number of utterances = 100
2020-06-03 17:17:12,045 (splitjson:43) INFO: /usr/bin/python3 /content/espnet/egs/an4/asr1/../../../utils/splitjson.py --parts 8 dump/test/deltafalse/data.json
2020-06-03 17:17:12,047 (splitjson:55) INFO: number of utterances = 134
2020-06-03 17:29:18,702 (concatjson:36) INFO: /usr/bin/python3 /content/espnet/egs/an4/asr1/../../../utils/concatjson.py exp/train_nodev_pytorch_train_mtlalpha0.5/decode_train_dev_decode_ctcweight0.5_batchsize0_lm_word100/data.1.json exp/train_nodev_pytorch_train_mtlalpha0.5/decode_train_dev_decode_ctcweight0.5_batchsize0_lm_word100/data.2.json exp/train_nodev_pytorch_train_mtlalpha0.5/decode_train_dev_decode_ctcweight0.5_batchsize0_lm_word100/data.3.json exp

In [34]:
# 9.9 Insert your code below this line, before running this cell:
# First 2 results correspond to decode_ctcweigth0.5.yaml 
# Last 2 lines correspond to decode_ctcweigth1.0.yaml 
! echo train_mtlalpha0.5
! echo decode_test_decode_ctcweight0.5_batchsize0_lm_word100
!ls espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/result.txt | xargs -n1 grep -e goncalo -e ricardo -e SPK -m 3
! echo decode_test_decode_ctcweight1.0_batchsize0_lm_word100
!ls espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight1.0_batchsize0_lm_word100/result.txt | xargs -n1 grep -e goncalo -e ricardo -e SPK -m 3
! echo train_mtlalpha1.0
! echo decode_test_decode_ctcweight0.5_batchsize0_lm_word100
!ls espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha1.0/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/result.txt | xargs -n1 grep -e goncalo -e ricardo -e SPK -m 3
! echo decode_test_decode_ctcweight1.0_batchsize0_lm_word100
!ls espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha1.0/decode_test_decode_ctcweight1.0_batchsize0_lm_word100/result.txt | xargs -n1 grep -e goncalo -e ricardo -e SPK -m 3

train_mtlalpha0.5
decode_test_decode_ctcweight0.5_batchsize0_lm_word100
|   SPKR      |   # Snt     # Wrd    |   Corr        Sub        Del        Ins         Err      S.Err   |
|   goncalo   |      2        107    |   36.4       51.4       12.1       42.1       105.6      100.0   |
|   ricardo   |      2         65    |   40.0       23.1       36.9        0.0        60.0      100.0   |
decode_test_decode_ctcweight1.0_batchsize0_lm_word100
|   SPKR      |   # Snt     # Wrd    |   Corr        Sub        Del        Ins         Err      S.Err   |
|   goncalo   |      2        107    |   41.1       52.3        6.5       39.3        98.1      100.0   |
|   ricardo   |      2         65    |   36.9       24.6       38.5        0.0        63.1      100.0   |
train_mtlalpha1.0
decode_test_decode_ctcweight0.5_batchsize0_lm_word100
|   SPKR      |   # Snt     # Wrd    |   Corr        Sub        Del        Ins         Err      S.Err   |
|   goncalo   |      2        107    |   33.6       40.2    

In [33]:
# 9.10 Insert your code below this line, before running this cell:
! echo train_mtlalpha0.5
! echo decode_test_decode_ctcweight0.5_batchsize0_lm_word100
! grep -i -A 5 goncalo_rec1 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/result.txt
! grep -i -A 5 ricardo_rec1 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/result.txt
! grep -i -A 5 goncalo_rec2 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/result.txt
! grep -i -A 5 ricardo_rec2 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/result.txt
! echo decode_test_decode_ctcweight1.0_batchsize0_lm_word100
! grep -i -A 5 goncalo_rec1 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight1.0_batchsize0_lm_word100/result.txt
! grep -i -A 5 ricardo_rec1 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight1.0_batchsize0_lm_word100/result.txt
! grep -i -A 5 goncalo_rec2 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight1.0_batchsize0_lm_word100/result.txt
! grep -i -A 5 ricardo_rec2 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha0.5/decode_test_decode_ctcweight1.0_batchsize0_lm_word100/result.txt
! echo train_mtlalpha1.0
! echo decode_test_decode_ctcweight0.5_batchsize0_lm_word100
! grep -i -A 5 goncalo_rec1 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha1.0/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/result.txt
! grep -i -A 5 ricardo_rec1 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha1.0/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/result.txt
! grep -i -A 5 goncalo_rec2 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha1.0/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/result.txt
! grep -i -A 5 ricardo_rec2 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha1.0/decode_test_decode_ctcweight0.5_batchsize0_lm_word100/result.txt
! echo decode_test_decode_ctcweight1.0_batchsize0_lm_word100
! grep -i -A 5 goncalo_rec1 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha1.0/decode_test_decode_ctcweight1.0_batchsize0_lm_word100/result.txt
! grep -i -A 5 ricardo_rec1 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha1.0/decode_test_decode_ctcweight1.0_batchsize0_lm_word100/result.txt
! grep -i -A 5 goncalo_rec2 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha1.0/decode_test_decode_ctcweight1.0_batchsize0_lm_word100/result.txt
! grep -i -A 5 ricardo_rec2 espnet/egs/an4/asr1/exp/train_nodev_pytorch_train_mtlalpha1.0/decode_test_decode_ctcweight1.0_batchsize0_lm_word100/result.txt

train_mtlalpha0.5
decode_test_decode_ctcweight0.5_batchsize0_lm_word100
id: (goncalo-goncalo_rec1)
Scores: (#C #S #D #I) 12 5 0 39
REF:  * * * * ******* * * * ******* t * * ******* * * * * <space> * * * * A <space> s i x ******* * * * * <space> t * * H       R e * e * ******* * * <space> * * * * C <space> * * * D 
HYP:  F I V E <SPACE> T W O <SPACE> t W O <SPACE> N I N E <space> S E V E N <space> s i x <SPACE> F I V E <space> t W O <SPACE> S e V e N <SPACE> O H <space> E I G H T <space> F O U R 
Eval: I I I I I       I I I I         I I I       I I I I         I I I I S               I       I I I I           I I S       S   I   I I       I I         I I I I S         I I I S 

id: (ricardo-ricardo_rec1)
Scores: (#C #S #D #I) 7 4 8 0
REF:  s <space> L <space> T H R E e <space> E I G H T <space> B <space> B 
HYP:  s <space> O <space> * * * * e <space> * * * * N <space> E <space> E 
Eval:           S         D D D D           D D D D S         S         S 

id: (goncalo-goncalo_rec2)
Sco

## Thanks for your attention